# Minor help functions

In [ ]:
def printProgressEnum(index, length, label=None):
    print('\r{}Progress: {}/{} ({:.2f}%)'.
                  format(label if label != None else '', index + 1, length, 100 * (index + 1) / length), flush=True, end='')
    
def showTensorPicture(tensor_image, label=None):
    # img = tensor_image.permute(1, 2, 0)
    img = cv2.cvtColor(tensor_image.permute(1, 2, 0).numpy(), cv2.COLOR_BGR2RGB)
    plt.imshow(img)
    if label:
        plt.title(label)
    plt.show()
    

def UnmakeRel(coords, w, h):
    return list(map(int,
                    [coords[0] * w, coords[1] * h, coords[2] * w, coords[3] * h]
                   ))

def MakeRel(coords, w, h):
    return list(map(float,
                    [coords[0] / w, coords[1] / h, coords[2] / w, coords[3] / h]
                   ))

def ConvertAbsTLWH2CV2Rectangle(coords):
    return list(map(int, 
                    [coords[0], coords[1] , coords[0] + coords[2], coords[1] + coords[3]]
                   )
               )

def ConvertCenterXYWH2CV2Rectangle(coords):
    return list(map(int, 
                    [coords[0] - coords[2] / 2, coords[1] - coords[3] / 2, coords[0] + coords[2] / 2, coords[1] + coords[3] / 2]
                   )
               )

def ConvertCV2Rectangle2CenterXYWH(coords):
    return list(map(int,
                    [(coords[2] + coords[0]) / 2, 
                     (coords[3] + coords[1]) / 2,
                     coords[2] - coords[0],
                     coords[3] - coords[1],
                    ]
                   ))

# Learn specific

In [ ]:
def train_epoch(model, loader, loss_op, optim, device):
    loss_ = []
    torch.set_grad_enabled(True)
    
    # Таким образом переводим модель в режим обучения
    # В этом режиме вычисляются градиенты, нужные для обучения
    model.train()
    model.to(device)
    for batch_idx, (imgs_batch, labels_batch) in enumerate(loader):
        # print(imgs_batch.shape)
        imgs_batch = imgs_batch.to(device)
        # print(labels_batch)
        labels_batch = labels_batch.to(device)

        pred = model(imgs_batch)

        loss = loss_op(pred, labels_batch)
        # Сохраним в историю эпохи
        l_ = loss.item()
        loss_.append(l_)
        # model.addValidHistoryAcc(l_)
        # optim.zero_grad()
        for param in model.parameters(): # https://pytorch.org/tutorials/recipes/recipes/tuning_guide.html
            param.grad = None
        loss.backward()
        optim.step()
        
        if batch_idx % 20 == 0:
            print('\rTrain Epoch: {} [{}/{} ({:.2f}%)]'.
                  format(epoch, batch_idx * len(imgs_batch), len(loader.dataset),
                 100 * batch_idx * len(imgs_batch) / len(loader.dataset)), flush=True, end='')
        
    print('\rTrain Epoch: {} [{}/{} (100%)]     \n'.
                  format(epoch, len(loader.dataset), len(loader.dataset)), flush=True, end='')
    
    return np.mean(loss_)


from sklearn.metrics import accuracy_score

def evaluate_batch_accuracy(y_pred, y_true):
    '''
    Оценка точности предсказания (accuracy)

    y_pred:
        батч сырых степеней уверенности, размер (N, K)
    y_true:
        вектор истинных значений, размер (N)
    '''
    y_pred = y_pred.detach().numpy()
    y_true = y_true.detach().numpy()
    # print(y_true)
    # print(y_pred)
    accuracy = 0
    for i in range(len(y_true)):
        index_max = max(range(len(y_pred[i, :])), key=y_pred[i].__getitem__)
        # print(index_max)
        if (index_max == y_true[i]):
            accuracy += 1
    accuracy /= len(y_pred)
    return accuracy

def valid_epoch(model, loader, device):
    acc_ = []
    torch.set_grad_enabled(False)
    # Таким образом переводим модель в режим исполнения (inference)
    # В этом режиме отключены градиенты, он быстрее, 
    #   но в нём нельзя обучать модель
    model.eval()
    model.to(device)
    for batch_idx, (imgs_batch, labels_batch) in enumerate(loader):
        imgs_batch = imgs_batch.to(device)
        #print(labels_batch)
        labels_batch = labels_batch.to(device)
        
        pred = model(imgs_batch)
        
        local_acc = evaluate_batch_accuracy(pred.cpu(), labels_batch.cpu())
        acc_.append(local_acc)
        
        # model.addValidHistoryAcc(local_acc)
        
        if batch_idx % 20 == 0:
            print('\rValid Epoch: {} [{}/{} ({:.2f}%)]'.
                  format(epoch, batch_idx * len(imgs_batch), len(loader.dataset),
                 100 * batch_idx * len(imgs_batch) / len(loader.dataset)), flush=True, end='')
        
    print('\rValid Epoch: {} [{}/{} (100%)]     \n'.
                  format(epoch, len(loader.dataset), len(loader.dataset)), flush=True, end='')
    
    return np.mean(acc_)


# Verify specific

In [ ]:
import pandas as pd

def getRandomFromDataset(gt: pd.DataFrame, label='test'):
    gt = gt[gt['is_present'] == 1]
    random_instance = gt[gt['set']==label].sample(1)
    img_path = DATA_DIR / 'merged-rtsd' / random_instance['filename'].values[0]
    sign_class = random_instance['sign_class'].values[0]
    # print(img_path)
    img = cv2.imread(str(img_path))
    # print(img)
    img_NT = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    img = cv2.resize(img, (160, 160))
    img_T = torch.Tensor.permute(torch.Tensor(img), [2, 0, 1]).div(255)

        
    return img_NT, img_T.cuda(), sign_class, img_path

def getMaxIndex(t_arr):
    t_arr = t_arr.cpu().detach().numpy()
    return np.argmax(t_arr)

def getLabelFromModelOutput(t_arr):
    max_arg = getMaxIndex(t_arr)
    # print(max_arg)
    return MODEL_CLASS_UNMAP[max_arg]

def showImg(img, label=None):
    plt.imshow(img)
    if label:
        plt.title(label)
    plt.show()


# YOLO format converter